Before proceeding, ensure that:
- Python environment is activated in terminal (use "```.venv/Scripts/activate```" command)
- Python Notebook kernel is using Python environment.

In [1]:
# Install necessary libraries
!pip install ultralytics
!pip install -U ultralytics "ray[tune]"
!pip install roboflow
!pip install comet_ml
!pip install wandb
!pip install torch
!pip install torchvision
!pip install ipykernel
!pip install ipywidgets
!pip install python-dotenv

  Using cached comet_ml-3.49.8-py3-none-any.whl.metadata (4.1 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
Using cached comet_ml-3.49.8-py3-none-any.whl (726 kB)
Using cached rich-14.0.0-py3-none-any.whl (243 kB)
  Using cached wandb-0.19.9-py3-none-win_amd64.whl.metadata (10 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
Using cached wandb-0.19.9-py3-none-win_amd64.whl (20.2 MB)
Using cached GitPython-3.1.44-py3-none-any.whl (207 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)


In [2]:
# Check NVIDIA System Management Interface
!nvidia-smi

Sun Apr 20 21:00:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX230         WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   73C    P0            N/A  / 5001W |      83MiB /   2048MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
# Import necessary packages
import os
import comet_ml
import wandb
import torch
import ray
from ultralytics import YOLO
from ultralytics.utils import SETTINGS
from ray import tune, air
from roboflow import Roboflow
from dotenv import load_dotenv

In [10]:
# Set current working directory path
HOME = os.getcwd()
print(HOME)

# Load env variables
load_dotenv()

# Download Custom YOLO11 Barcode Dataset (use YOLOv8 format)
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace("bonitadagcutatimajo").project("yolo11-barcode-detection")
version = project.version(2)
dataset = version.download("yolov8-obb")

d:\USTP Files\4th Year\Thesis\HypTuning
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to YOLO11-Barcode-Detection-2 in yolov8-obb:: 100%|██████████| 4028/4028 [00:11<00:00, 359.59it/s]


In [14]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.6.0+cpu
None
False


In [ ]:
# Define search space
search_space = {
    "lr0": (1e-5, 1e-1),
    "lrf": (0.01, 1.0),
    "momentum": (0.6, 0.98),
    "weight_decay": (0.0, 0.001),
    "warmup_epochs": (0.0, 5.0),
    "warmup_momentum": (0.0, 0.95),
    "box": (0.02, 0.2),
    "cls": (0.2, 4.0)
}

# Run tuning
model = YOLO("yolo11n-obb.pt")
model.to("cuda")
model.tune(
    data=f"{dataset.location}/data.yaml",
    space=search_space,
    epochs=15,
    batch=32,
    imgsz=640,
    fraction=0.25,
    iterations=100,
    optimizer="AdamW",
    plots=True,
    save=False,
    val=False
)